# DSPy MCP Client - Intelligent Agent Integration

Learn how to connect **DSPy agents** to **MCP servers** for powerful tool integration.

**What you'll learn:**
- Connect DSPy to MCP servers
- Define DSPy signatures for complex tasks
- Use ReAct agents with MCP tools
- Chain multiple agent calls together

**Use Case:** WhatsApp automation - summarize messages and send intelligent replies

## Quick DSPy Recap

**DSPy** is a declarative framework for programming with language models. Unlike imperative approaches (manually crafting prompts), DSPy lets you define **what** you want, not **how** to prompt for it.

**Key Concepts:**

1. **Signatures**: Input/output specifications that tell the LLM what task to perform
   - Think of them as type-safe function signatures for LLMs
   - Example: `question: str -> answer: str`

2. **ReAct (Reasoning + Acting)**: An agent pattern that combines:
   - **Reasoning**: LLM thinks about what to do next
   - **Acting**: LLM calls tools to gather information
   - **Observing**: LLM sees tool results and decides next step
   - Loops until task is complete or max iterations reached

3. **MCP Integration**: Why DSPy + MCP is powerful
   - DSPy provides declarative agent logic
   - MCP provides reusable, standardized tools
   - Together: Build intelligent agents without manual prompt engineering

In this notebook, we'll see how DSPy makes MCP client code cleaner and more maintainable.

In [18]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

import dspy
from dotenv import load_dotenv
import os

load_dotenv()  # loads .env file

openai_api_key = os.getenv("OPENAI_API_KEY")

# Create server parameters for stdio connection
server_params = StdioServerParameters(
    command="/Users/afmjoaa/.local/bin/uv",
    args=[
        "--directory",
        "/Users/afmjoaa/PycharmProjects/agents/whatsapp-mcp/whatsapp-mcp-server",
        "run",
        "main.py",
    ],  # Optional command line arguments
    env=None,  # Optional environment variables
)


class GetLastFiveMessage(dspy.Signature):
    chat_number: str = dspy.InputField(
        desc="Whatsapp sender_phone_number from where the last 5 messages are to be fetched"
    )
    summary: str = dspy.OutputField(
        desc="Create a summary of the last 5 messages from the given chat number."
    )


class SendSummary(dspy.Signature):
    sender_phone_number: str = dspy.InputField(
        desc="Whatsapp sender_phone_number where the reply will be sent"
    )
    summary: str = dspy.InputField(
        desc="Summary of the last 5 messages from the given sender phone number."
    )
    reply: str = dspy.OutputField(desc="Generate a reply based on the summary.")


lm = dspy.LM(
    model="openai/gpt-4o-mini",  # openai/gpt-4o-mini
    api_key=openai_api_key,
)
dspy.configure(lm=lm)


async def run(chat_number):
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection

            await session.initialize()
            # List available tools
            tools = await session.list_tools()

            # Convert MCP tools to DSPy tools
            dspy_tools = []
            for tool in tools.tools:
                dspy_tools.append(dspy.Tool.from_mcp_tool(session, tool))

            # Create the agent
            react = dspy.ReAct(GetLastFiveMessage, tools=dspy_tools, max_iters=2)
            result = await react.acall(chat_number=chat_number)
            print(result)

            # generate reply and send
            reply = dspy.ReAct(
                SendSummary.with_instructions(
                    instructions="Generate the reply and send the reply to the sender phone number"
                ),
                tools=dspy_tools,
                max_iters=2,
            )
            final_result = await reply.acall(sender_phone_number=chat_number, summary=result.summary)
            print(final_result)


if __name__ == "__main__":
    import asyncio
    await run("8801716366412")


/Users/afmjoaa/PycharmProjects/agents/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Prediction(
    trajectory={'thought_0': 'To gather information for the summary, I need to fetch the last 5 messages from the specified chat number. I will proceed to list messages from this chat.', 'tool_name_0': 'list_messages', 'tool_args_0': {'sender_phone_number': '8801716366412', 'limit': 5}, 'observation_0': "[2026-01-11 12:37:41] Chat: 8801716366412 From: 8801716366412: It eats flies and jumps around all day\n[2026-01-11 12:37:46] Chat: 8801716366412 From: 8801716366412: It's big\n[2026-01-17 14:54:21] Chat: 8801716366412 From: Me: Chuchunii\n[2026-01-11 12:37:27] Chat: 8801716366412 From: 8801716366412: It is green and grumpy\n[2026-01-11 12:37:41] Chat: 8801716366412 From: 8801716366412: It eats flies and jumps around all day\n[2026-01-11 12:37:46] Chat: 8801716366412 From: 8801716366412: It's big\n[2026-01-11 12:37:19] Chat: 8801716366412 From: 8801716366412: Do you think it has a house inside the pond?\n[2026-01-11 12:37:27] Chat: 8801716366412 From: 8801716366412: It is gr

/Users/afmjoaa/PycharmProjects/agents/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Prediction(
    trajectory={'thought_0': 'Given the playful nature of the previous discussion about the frog, it seems appropriate to continue the conversation with a whimsical reply. I should send a message that maintains the lighthearted tone.', 'tool_name_0': 'send_message', 'tool_args_0': {'recipient': '8801716366412', 'message': 'I wonder if the grumpy frog has any friends in the pond! Perhaps a cheerful turtle or a wise old fish could join our ribbiting tales?'}, 'observation_0': '{"success": true, "message": "Message sent to 8801716366412"}', 'thought_1': 'The whimsical message about the grumpy frog has been sent successfully. Given the lighthearted nature of the conversation, I could either wait for a response or check for any previous interactions for context. However, I believe engaging further after the last message is not immediately necessary, so I can mark this task as complete.', 'tool_name_1': 'finish', 'tool_args_1': {}, 'observation_1': 'Completed.'},
    reasoning='T

## Key Takeaways

### What You Learned

**1. DSPy + MCP Integration**
- DSPy provides **declarative agent logic** through Signatures and ReAct
- MCP provides **reusable, standardized tools** via the protocol
- Integration via `dspy.Tool.from_mcp_tool()` - automatic conversion
- One MCP server can serve many DSPy agents

**2. DSPy Signatures**
- Define **what** the LLM should do, not **how** (no prompt engineering)
- `InputField` = what the LLM receives (context, data, questions)
- `OutputField` = what the LLM produces (answers, summaries, decisions)
- Descriptive `desc` parameters guide the LLM's understanding

**3. ReAct Agents**
- **Reasoning**: LLM thinks about the task
- **Acting**: LLM calls tools to gather information
- **Observing**: LLM sees results and decides next step
- `max_iters` prevents infinite loops (2-5 is typical)

**4. Agent Chaining**
- Output from one agent → Input to another agent
- Each agent has its own signature and reasoning loop
- Agents share the same tool set from MCP server
- Pattern: `result2 = await agent2.acall(input=result1.output)`

**5. Async/Await Patterns**
- MCP connections are async (network I/O)
- Use `async with` for proper resource cleanup
- Jupyter notebooks support top-level `await`
- Always properly close MCP sessions

### Comparison: DSPy vs LangGraph vs Vanilla

| Aspect                 | Vanilla        | DSPy                 | LangGraph          |
|------------------------|----------------|----------------------|--------------------|
| **Abstraction Level**  | Low            | Medium               | High               |
| **Declarative**        | No             | Yes                  | Yes                |
| **Tool Integration**   | Manual JSON    | Semi-automatic       | Automatic          |
| **Agent Pattern**      | Custom loops   | ReAct built-in       | Graph-based        |
| **State Management**   | Manual         | Automatic            | Graph nodes        |
| **Prompt Engineering** | Manual         | Optimizable          | Framework-handled  |
| **Learning Curve**     | Easy           | Medium               | Steep              |
| **Best For**           | Simple scripts | Research/prototyping | Production systems |

## ReAct Agent Basics

**ReAct** (Reasoning + Acting) is a powerful agent pattern that combines:
- **Thought**: LLM reasons about what to do next
- **Action**: LLM calls a tool to gather information
- **Observation**: LLM observes the tool's result
- **Loop**: Repeat until task is complete or max_iters reached

**Creating a ReAct Agent:**
```python
agent = dspy.ReAct(
    signature,        # What task to perform
    tools=dspy_tools, # What tools are available
    max_iters=2       # Maximum reasoning-acting loops
)
```
![Function calling translation diagram](../assets/react_agent.png)


## Converting MCP Tools to DSPy Format

DSPy needs tools in a specific format to use them with ReAct agents. The conversion process:

1. **Connect to MCP server** via stdio client
2. **Initialize session** with the server
3. **List available tools** from the server
4. **Convert each MCP tool** to DSPy format using `dspy.Tool.from_mcp_tool()`

This conversion happens automatically - DSPy handles the protocol details for us.

In [ ]:
# Signature 1: Get and summarize last 5 messages
class GetLastFiveMessage(dspy.Signature):
    """Fetch the last 5 messages from a WhatsApp chat and create a summary."""
    
    chat_number: str = dspy.InputField(
        desc="WhatsApp sender_phone_number from where the last 5 messages are to be fetched"
    )
    summary: str = dspy.OutputField(
        desc="Create a summary of the last 5 messages from the given chat number."
    )

print("✓ GetLastFiveMessage signature defined")
print("  - Input: chat_number (phone number)")
print("  - Output: summary (summary of last 5 messages)")

## DSPy Signatures

**Signatures** are DSPy's way of defining what an LLM should do. Think of them as type-safe contracts:
- **InputField**: What the LLM receives (context, question, data)
- **OutputField**: What the LLM should produce (answer, summary, decision)

The `desc` parameter is critical - it guides the LLM on how to use each field.

We'll define two signatures for our WhatsApp automation workflow:
1. **GetLastFiveMessage**: Fetch and summarize recent messages
2. **SendSummary**: Generate a reply based on the summary